## Load in data and remove extra columns ##

In [55]:
import pandas as pd
import glob
import os

from datetime import datetime

path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))

for file in csv_files:
    df = pd.read_csv(file)

historic = False
if 'Date' in df:
    historic = True
    df = df[['Date', 'Last', 'First', 'Score', 'Sleep Hours']]
else:
    df = df[['Last', 'First', 'Readiness Score', 'Sleep']]

df.head()

,Last,First,Readiness Score,Sleep
0,Dellaperuta,Talia,72.87,9.0
1,Wrigley,Lauren,74.20,9.0
2,Dellaperuta,Tori,74.69,9.0
3,Colton,Emily,86.75,8.0
4,Oliaro,Mia,87.75,8.5


## Add rows with week averages ##

In [56]:
if historic:
    dates = df['Date'].unique()
    weeks = []
    week = []

    # Separate days into weeks (Mon-Sun)
    for date in dates:
        clean_date = datetime.strptime(date, "%Y-%m-%d")
        # Start of first full week
        if clean_date.weekday() == 6:
            week = []
            week.append(date)
        # Add day to first full week
        if len(week) != 0 and clean_date.weekday() != 6:
            week.append(date)
        # Add week to weeks on last day of full week
        if len(week) != 0 and clean_date.weekday() == 0:
            weeks.append(week)

    # Add row with team averages for each full week
    for week in weeks:
        score_mean = df.loc[df['Date'].isin(week), 'Score'].mean()
        sleep_mean = df.loc[df['Date'].isin(week), 'Sleep Hours'].mean()
        row = ({
            'Date': f"Week of {week[-1][5:10]} to {week[0][5:10]}",
            'Last': 'Team',
            'First': 'Average',
            'Score': score_mean,
            'Sleep Hours': sleep_mean
        })
        idx = df.loc[df['Date'] == week[0]].index.array[0]
        df2 = pd.DataFrame(row, index=[idx])
        df = pd.concat([df, df2]).sort_index().reset_index(drop=True)
    df['Score'] = df['Score'].round(2)
    df['Sleep Hours'] = df['Sleep Hours'].round(1)
df.head(50)
    



,Last,First,Readiness Score,Sleep
0,Dellaperuta,Talia,72.87,9.0
1,Wrigley,Lauren,74.20,9.0
2,Dellaperuta,Tori,74.69,9.0
3,Colton,Emily,86.75,8.0
4,Oliaro,Mia,87.75,8.5
5,Faasse,Kate,88.50,8.0
6,Dellarose,Tessa,89.00,8.0
7,Sentnor,Allyson,91.00,10.0
8,Moxley,Emily,93.75,9.0
9,Dahlien,Maddie,94.00,9.0


## Add conditional formatting and export to excel ##

In [57]:
if historic:
    score = 'Score'
    sleep = 'Sleep Hours'
else:
    score = 'Readiness Score'
    sleep = 'Sleep'

# Date conditional formatting
if historic:
    def date(df):
        highlight = ''
        if int(df['Date'][-1]) % 2 == 0:
            highlight = 'background-color: #D59CFF'
        else:
            highlight = 'background-color: #EFDAFF'
        
        if df['Date'][0] == 'W':
            highlight = 'background-color: #7BAFD4'
        return [highlight]

# Score conditional formatting
def score_f(df):
    highlight = ''
    if df[score] < 60:
        highlight = 'background-color: red'
    elif df[score] >= 60 and df[score] < 70:
        highlight = 'background-color: orange'
    elif df[score] >= 70 and df[score] < 85:
        highlight = 'background-color: yellow'
    elif df[score] >= 85:
        highlight =  'background-color: #15DE00'
    return [highlight]

# Sleep conditional formatting
def sleep_f(df):
    highlight = ''
    if df[sleep] < 6:
        highlight = 'background-color: red'
    elif df[sleep] >= 6 and df[sleep] < 8:
        highlight = 'background-color: yellow'
    elif df[sleep] >= 8:
        highlight =  'background-color: #15DE00'
    return [highlight]

new_df = (df
    .style
    .apply(score_f, subset=[score], axis=1)
    .apply(sleep_f, subset=[sleep], axis=1)
    )

if historic:
    new_df = (new_df.apply(date, subset=['Date'], axis=1))

new_df.to_excel('WSoccer_Readiness.xlsx', index=False)